In [ ]:
# default_exp cli

# CLI tools
> Command line tools

In [ ]:
#export
from pathlib import Path
from fastscript import call_parse, Param
from geoget.download import *

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
from IPython.core.debugger import set_trace

In [ ]:
#export
_repChoices=['ALBERS', 'GEO', 'LAMAZ', 'MERCAT', 'PS', 'ROBIN', 'SNSOID', 'TM', 'UTM']

@call_parse
def geoget_ladsweb(
    product:Param("Name of the product", str), 
    collection:Param("Collection number", str), 
    tstart:Param("Start of serach window yyyy-mm-dd HH:MM:SS", str), 
    tend:Param("End of search windo yyyy-mm-dd HH:MM:SS", str),
    bbox:Param("Bounding box in format left bottom right top", list), 
    email:Param("ladsweb user email", str)=None,
    auth:Param("ladsweb user authentication key (go to Profile>App Keys)", str)=None,
    path_save:Param("Path to save the outputs of the request", str)=None,
    bands:Param("List of bands to download", list)=None,
    coordsOrTiles:Param("coordsOrTiles parameter", str, choices=["coords", "tiles"])="coords", 
    daynight:Param("Select images for Day, Night or both", str, choices=['D', 'N', 'DNB'])="DNB",
    repName:Param("Reprojection type", str, choices=_repChoices)='GEO', 
    repPixSize:Param("Pixel size in units depending on the reprojection type", float)=0.01, 
    repResample:Param("Resampling method", str, choices=['bilinear', 'nearest'])='bilinear',
    doMosaic:Param("",str)='False',
    searchOnly:Param("Set to true just to search for file ids", bool)=False):
    
    kwargs = {key: value for key, value in locals().items()}
    lads = Ladsweb(**kwargs)
    ids = lads.search_files()
    if len(ids) == 0: 
        warning.warn("No files found", UserWarning)
        return
    if searchOnly: return ids
    if email is None: raise Exception("The `email` is required to place an order")
    if bands is None: raise Exception("A list of `bands` is not defined")
    orderId = lads.send_order(ids, email)
    try:
        while status != 'Available':
            status = order_status(orderId)
            print(f"Current status: {status}")
            sleep(60)
    except KeyboardInterrupt:
        raise Exception("KeaybordInterrupt")
    if auth is None: raise Exception("The `auth` key is required to download the files")
    if path_save is None: raise Exception("The `path_save` is not defined")
    Path(path_save).mkdir(exist_ok=True, parents=True)
    status = lads.download_files(order_id, path_save, auth)
    if status: result = lads.release_order(order_id, email)
    if result: print(f'Process complete, files saved at {path_save}.')

In [ ]:
#hide
notebook2script()

Converted 00_download.ipynb.
Converted 01_cli.ipynb.
Converted index.ipynb.
